In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# Set the paths to the dataset
train_data_dir = 'NLM Augmentation/train'
test_data_dir = 'NLM Augmentation/test'

# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 16

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1. / 255)

# Load the pretrained model for feature extraction
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the base model's layers
base_model.trainable = False

# Define the number of classes based on your dataset
num_classes = 7

# Transformer-based feature extraction
inputs = Input(shape=(img_width, img_height, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Specify the number of training and validation steps
train_steps = train_generator.samples // batch_size
val_steps = test_generator.samples // batch_size

# First part of training (5 epochs)
history_1 = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=5,
    validation_data=test_generator,
    validation_steps=val_steps,
    callbacks=[EarlyStopping(patience=3)]
)


# Save the trained model after the first part of training
model.save('transformer_model_part1')   

# Display the history from the first part of training
print("History from the first part (epochs 1 to 5):")
print(history_1.history)


Found 6951 images belonging to 7 classes.
Found 3001 images belonging to 7 classes.
Epoch 1/5
434/434 [==============================] - 538s 1s/step - loss: 1.9099 - accuracy: 0.1945 - val_loss: 1.8961 - val_accuracy: 0.2129
Epoch 2/5
434/434 [==============================] - 483s 1s/step - loss: 1.9096 - accuracy: 0.2033 - val_loss: 1.9037 - val_accuracy: 0.2129
Epoch 3/5
434/434 [==============================] - 492s 1s/step - loss: 1.9075 - accuracy: 0.1941 - val_loss: 1.9224 - val_accuracy: 0.1574
Epoch 4/5
434/434 [==============================] - 485s 1s/step - loss: 1.9029 - accuracy: 0.2000 - val_loss: 1.9003 - val_accuracy: 0.1975
INFO:tensorflow:Assets written to: transformer_model_part1\assets


INFO:tensorflow:Assets written to: transformer_model_part1\assets


History from the first part (epochs 1 to 5):
{'loss': [1.909897804260254, 1.9096297025680542, 1.9075415134429932, 1.9029009342193604], 'accuracy': [0.19452054798603058, 0.20331650972366333, 0.194087952375412, 0.20000000298023224], 'val_loss': [1.8961410522460938, 1.9036762714385986, 1.9223660230636597, 1.9003325700759888], 'val_accuracy': [0.2129010707139969, 0.2129010707139969, 0.15741978585720062, 0.1975267380475998]}


In [1]:
# Load the saved model from Cell 1
loaded_model = tf.keras.models.load_model('transformer_model_part1')

# Unfreeze the base model's layers
base_model = loaded_model.layers[1]
base_model.trainable = True


# Compile the model after unfreezing
loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
 

# Second part of training (5 additional epochs)
history_2 = loaded_model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=10,  # Train for 5 additional epochs (epochs 6 to 10)
    initial_epoch=5,  # Start from epoch 6
    validation_data=test_generator,
    validation_steps=val_steps,
    callbacks=[EarlyStopping(patience=3)]
)

 
# Display the history from the second part of training
print("History from the second part (epochs 6 to 10):")
print(history_2.history)

# Get the outputs of the global average pooling layer
feature_outputs = model.get_layer('global_average_pooling2d').output

# Create a new model with the feature outputs as the output layer
feature_extraction_model = Model(inputs=model.input, outputs=feature_outputs)

# Extract features from the train and test data
train_features = feature_extraction_model.predict(train_generator)
test_features = feature_extraction_model.predict(test_generator)

# Save the extracted features
train_features_path = 'Feature_Extraction/train/train_features.npy'
test_features_path = 'Feature_Extraction/test/test_features.npy'

np.save(train_features_path, train_features)
np.save(test_features_path, test_features)


NameError: name 'tf' is not defined

In [ ]:
import numpy as np
train_features = np.load('Feature_Extraction/train/train_features.npy')
test_features = np.load('Feature_Extraction/test/test_features.npy')

# Convert the feature arrays to lists
train_features_list = train_features.tolist()
test_features_list = test_features.tolist()

# Print the number of images and the number of features extracted per image
print("Number of images in the training set:", len(train_features_list))
print("Number of images in the test set:", len(test_features_list))
print("Number of features extracted per image:", len(train_features_list[0]))